# Allow system to have an in memory saver
This help to keep the conversation going. The system remembers previous messages
The drawback is that when you log out of the application, the application will not remember previous conversation.
You will need and sqlite database implementation to achieve this

In [2]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv

In [3]:
load_dotenv()
# llm = ChatGroq(model="llama3-8b-8192")   # 8b model  - faster model
llm = ChatGroq(model="llama3-70b-8192")  # 70b model  x8 better but slower

## Create the schema and node

In [4]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state:BasicChatState)->BasicChatState:
    return {
        "messages": [llm.invoke(state["messages"])]  #
    }

### Create the graph with Memory chechpoint

In [10]:
graph = StateGraph(BasicChatState)

graph.add_node("chatbot", chatbot)
graph.set_entry_point("chatbot")

graph.add_edge("chatbot", END)

# add memory
memory= MemorySaver()
app = graph.compile(checkpointer=memory)

#tie the memory with a thread ID. A unique id that shows the conversation history as it builds
config = {"configurable":{
    "thread_id": 1
}}

### Get the Human question and ensure you tie it to a memory thread

In [11]:
while True:
    user_input = input("User: ")

    if user_input in ["exit","end"]:
        break 
    # remember to pass the configuration, thread id to the invoke method
    result = app.invoke({
        "messages":[HumanMessage(content=user_input)]
    }, config=config)
    # print(result)
    print("AI:",result["messages"][-1].content)


User:  Hello, My name is Emmanuel


AI: Hello Emmanuel! It's nice to meet you. Is there something I can help you with, or would you like to chat?


User:  can you tell me the current president of Nigeria


AI: The current President of Nigeria is Muhammadu Buhari. He has been in office since May 29, 2015. Would you like to know more about him or Nigerian politics in general?


User:  add 3 +5


...

User:  multiply it by 10


AI: 8 × 10 = 80


User:  what is my name again?


AI: Your name is Emmanuel.


User:  exit


In [13]:
print(app.get_state(config))

StateSnapshot(values={'messages': [HumanMessage(content='Hello, My name is Emmanuel', additional_kwargs={}, response_metadata={}, id='6c57572b-69d9-4ba6-b66a-9da6484281d2'), AIMessage(content="Hello Emmanuel! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 16, 'total_tokens': 43, 'completion_time': 0.077142857, 'prompt_time': 0.000194767, 'queue_time': 0.093280354, 'total_time': 0.077337624}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--905d6e04-9c36-49ea-9584-cb14ba3be893-0', usage_metadata={'input_tokens': 16, 'output_tokens': 27, 'total_tokens': 43}), HumanMessage(content='can you tell me the current president of Nigeria', additional_kwargs={}, response_metadata={}, id='3bf090f5-044c-46f9-bb65-6330b8e1542e'), AIMessage(content='The current President of Nigeria is 